#1. Cài đặt thư viện cần thiết vào mô trường

In [1]:
%pip install -q -U transformers accelerate datasets bitsandbytes trl unsloth
%pip install -q -U evaluate rouge_score bert_score sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00


#2. Tải mô hình cần huấn luyện

## sử dụng mô hình unsloth/llama-3.1-8b-bnb-4bit

In [2]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/llama-3.1-8b-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Tải mô hình và tokenizer

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.9.4: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## Cấu hình LoRA

In [4]:

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Unsloth 2025.9.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


#3. tải dataset cho huấn luyện

In [5]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import evaluate
from tqdm.auto import tqdm

## Tải và định dạng dataset ViHealthQA

In [6]:
print("Đang tải dataset tarudesu/ViHealthQA...")
dataset1 = load_dataset("tarudesu/ViHealthQA")

Đang tải dataset tarudesu/ViHealthQA...


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

val.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7009 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/993 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2013 [00:00<?, ? examples/s]

## Hiển thị một ví dụ từ tập huấn luyện để xem định dạng dữ liệu.

In [7]:
print(dataset1['train'][0])

{'id': 1, 'question': 'Đang chích ngừa viêm gan B có chích ngừa Covid-19 được không?', 'answer': 'Nếu anh/chị đang tiêm ngừa vaccine phòng bệnh viêm gan B, anh/chị vẫn có thể tiêm phòng vaccine phòng Covid-19, tuy nhiên vaccine Covid-19 phải được tiêm cách trước và sau mũi vaccine viêm gan B tối thiểu là 14 ngày.', 'link': 'https://vnexpress.net/tu-van-tiem-vaccine-covid-19-an-toan-hieu-qua-p57'}


## Định nghĩa một hàm để định dạng mỗi mẫu dữ liệu
(cặp câu hỏi-câu trả lời) thành một chuỗi văn bản duy nhất theo định dạng prompt cho mô hình. Sau đó áp dụng hàm này cho toàn bộ tập huấn luyện.

In [8]:
def format_instruction_vihealthqa(sample):
    return f"""### Question:\n{sample['question']}\n\n### Answer:\n{sample['answer']}EOS"""
formatted_dataset1 = dataset1['train'].map(lambda sample: {'text': format_instruction_vihealthqa(sample)}, remove_columns=dataset1['train'].column_names)


Map:   0%|          | 0/7009 [00:00<?, ? examples/s]

# 4. **Fine_tuning_Llama_với_ViHealthQA**

## Chia tập dữ liệu ViHealthQA thành train và test

## Chia tập dữ liệu đã định dạng thành tập huấn luyện (90%) và tập kiểm tra (10%) để sử dụng trong quá trình huấn luyện và đánh giá.

In [9]:

train_test_split1 = formatted_dataset1.train_test_split(test_size=0.1)
train_dataset1 = train_test_split1['train']
test_dataset1 = train_test_split1['test']


## Khởi tạo SFTTrainer từ thư viện trl.
SFTTrainer là một công cụ tiện lợi để fine-tuning các mô hình ngôn ngữ bằng phương pháp Supervised Fine-Tuning. Cấu hình các tham số huấn luyện như kích thước batch, tốc độ học, số bước huấn luyện, v.v.

In [16]:
# Khởi tạo SFTTrainer cho dataset ViHealthQA
trainer1 = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset1,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1500,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs_vihealthqa",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)

## Bắt đầu quá trình fine-tuning mô hình trên tập dữ liệu huấn luyện đã chuẩn bị.
Trainer sẽ tự động quản lý quá trình huấn luyện, bao gồm forward pass, backward pass và cập nhật trọng số LoRA.

In [ ]:
print("Bắt đầu fine-tuning trên dataset ViHealthQA...")
trainer1.train()

Bắt đầu fine-tuning trên dataset ViHealthQA...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,308 | Num Epochs = 2 | Total steps = 1,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,1.850300
2,1.651000
3,1.777000
4,1.769800
5,1.651200
6,1.585000
7,1.789600
8,1.863000
9,1.686400
10,1.724700


## Hiển thị Biểu đồ Loss của quá trình Fine-tuning

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

# Đường dẫn đến file log do SFTTrainer tạo ra
log_file = "outputs_vihealthqa/checkpoint-60/trainer_state.json"

# Kiểm tra xem file log có tồn tại không
if not os.path.exists(log_file):
    print("Không tìm thấy file log. Vui lòng chạy cell fine-tuning trước.")
else:
    # Đọc dữ liệu từ file log
    with open(log_file, "r") as f:
        trainer_state = json.load(f)

    # Trích xuất dữ liệu loss và steps
    log_history = trainer_state["log_history"]
    steps = [log["step"] for log in log_history if "loss" in log]
    train_losses = [log["loss"] for log in log_history if "loss" in log]

    # Vẽ biểu đồ loss
    plt.figure(figsize=(10, 6))
    plt.plot(steps, train_losses, label="Training Loss", marker='o')
    plt.title("Biểu đồ Loss trong quá trình Fine-tuning")
    plt.xlabel("Bước huấn luyện (Steps)")
    plt.ylabel("Giá trị Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

    print("Đã tạo biểu đồ Loss thành công.")

## Sau khi fine-tuning, đánh giá hiệu năng của mô hình trên tập kiểm tra.
Thu thập các câu trả lời được tạo ra bởi mô hình cho các câu hỏi trong tập kiểm tra.

In [ ]:
# Đánh giá mô hình sau lần fine-tuning đầu tiên
print("\nĐang đánh giá mô hình trên dataset ViHealthQA...")
predictions1 = []
references1 = []

for sample in tqdm(test_dataset1):
    prompt = sample['text'].split('### Answer:')[0]
    reference_answer = sample['text'].split('### Answer:')[1]

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_answer = generated_text.split('### Answer:')[1]

    predictions1.append(generated_answer)
    references1.append([reference_answer])


# 5. Đánh giá

## Tính toán các chỉ số đánh giá
 BLEU, ROUGE và BERTScore để đo lường sự tương đồng giữa câu trả lời của mô hình và câu trả lời tham chiếu.

In [ ]:
bleu_metric = evaluate.load("bleu")
bleu_results1 = bleu_metric.compute(predictions=predictions1, references=references1)
rouge_metric = evaluate.load("rouge")
rouge_results1 = rouge_metric.compute(predictions=predictions1, references=references1)
bertscore_metric = evaluate.load("bertscore")
bertscore_results1 = bertscore_metric.compute(predictions=predictions1, references=references1, lang="vi", device="cuda" if torch.cuda.is_available() else "cpu")

## In ra kết quả đánh giá cho các chỉ số BLEU, ROUGE và BERTScore.

In [ ]:
print("\nKết quả đánh giá trên ViHealthQA:")
print("BLEU:", bleu_results1['bleu'])
print("ROUGE:", rouge_results1)
print("BERTScore:", bertscore_results1)

## Trực quan hóa các Chỉ số Đánh giá

In [ ]:
metrics = ["BLEU", "ROUGE-1", "ROUGE-2", "ROUGE-L", "BERTScore_F1"]
scores = [bleu_score, rouge_1, rouge_2, rouge_l, bertscore_f1]

plt.figure(figsize=(10, 6))
plt.bar(metrics, scores, color=['blue', 'green', 'red', 'purple', 'orange'])
plt.title("Biểu đồ so sánh các chỉ số đánh giá")
plt.xlabel("Chỉ số")
plt.ylabel("Điểm số")
plt.grid(axis='y', linestyle='--')
plt.show()

## Trực quan hóa Phân bố điểm BERTScore

In [ ]:
if bertscore_f1_scores:
    plt.figure(figsize=(12, 6))
    plt.hist(bertscore_f1_scores, bins=10, edgecolor='black', alpha=0.7)
    plt.title("Phân bố điểm BERTScore F1 trên tập dữ liệu kiểm tra")
    plt.xlabel("Điểm BERTScore F1")
    plt.ylabel("Số lượng câu")
    plt.grid(True, linestyle='--')
    plt.show()
    print("Đã tạo biểu đồ phân bố BERTScore F1 thành công.")
else:
    print("Không có dữ liệu BERTScore để tạo biểu đồ.")

# 6. test và lưu trữ

## Lưu trữ adapter LoRA đã huấn luyện vào một thư mục.
Adapter này chứa các trọng số nhỏ đã học được trong quá trình fine-tuning và có thể được kết hợp với mô hình gốc để sử dụng.

In [ ]:
# Lưu adapter LoRA đã huấn luyện vào thư mục 'lora_model'
model.save_pretrained("lora_model")

# (Tùy chọn) Lưu cả tokenizer đi kèm
tokenizer.save_pretrained("lora_model")

## Môi trường có GPU

### Nén thư mục chứa adapter LoRA (tùy chọn)

In [ ]:
!zip -r lora_model.zip lora_model

### Tải file .zip chứa adapter LoRA về máy tính local của bạn. (tùy chọn)

In [ ]:
from google.colab import files
files.download('lora_model.zip')

### test với mô trường có GPU

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import os

# Tên thư mục nơi bạn đã lưu adapter LoRA
# Dựa trên file notebook của bạn, thư mục này có tên là 'lora_model'.
adapter_dir = "lora_model"

# Tên mô hình gốc mà bạn đã sử dụng để huấn luyện
model_name = "unsloth/llama-3.1-8b-bnb-4bit"

# Kiểm tra xem GPU có sẵn không
if not torch.cuda.is_available():
    print("Vui lòng chọn môi trường chạy có GPU (ví dụ: Google Colab với GPU T4).")
else:
    print("GPU đã sẵn sàng. Bắt đầu tải mô hình...")

    # Tải mô hình gốc và tokenizer
    try:
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_name,
            max_seq_length=2048,
            dtype=None,
            load_in_4bit=True,
        )
    except NotImplementedError:
        print("Lỗi: Unsloth chỉ hoạt động với GPU NVIDIA hoặc Intel. Đảm bảo bạn đã chọn GPU phù hợp trong cài đặt runtime.")
        model = None

    if model:
        # Cấu hình LoRA (cần thiết để tải lại adapter)
        model = FastLanguageModel.get_peft_model(
            model,
            r=16,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                              "gate_proj", "up_proj", "down_proj",],
            lora_alpha=16,
            lora_dropout=0,
            bias="none",
            use_gradient_checkpointing="unsloth",
            random_state=3407,
            max_seq_length=2048,
        )

        # Định dạng câu hỏi cho mô hình
        def create_prompt(question):
            return f"""### Question:
{question}

### Answer:
"""

        # Vòng lặp hỏi đáp
        print("\nBắt đầu trò chuyện với mô hình. Gõ 'exit' để thoát.")
        while True:
            question = input("Bạn có câu hỏi nào về sức khỏe không? ")
            if question.lower() == 'exit':
                break

            prompt = create_prompt(question)

            # Mã hóa câu hỏi và tạo câu trả lời
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=512,
                    use_cache=True,
                    pad_token_id=tokenizer.eos_token_id,
                    do_sample=True,
                    top_k=50,
                    top_p=0.95
                )

            # Giải mã câu trả lời và in ra màn hình
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Xử lý trường hợp mô hình trả lời theo định dạng khác nhau
            if '### Answer:' in generated_text:
                generated_answer = generated_text.split('### Answer:')[1].strip().replace('EOS', '')
            elif '### Output:' in generated_text:
                generated_answer = generated_text.split('### Output:')[1].strip().replace('EOS', '')
            else:
                generated_answer = generated_text.strip()

            print("\n[Mô hình trả lời]:")
            print(generated_answer)
            print("-" * 50)

## Môi trường không có GPU

### Xuất mô hình đã fine-tuning (adapter LoRA kết hợp với mô hình gốc) sang định dạng GGUF. (tùy chọn)
Định dạng này phù hợp để chạy mô hình trên các thiết bị khác nhau bằng các thư viện như llama.cpp hoặc ctransformers.

In [ ]:
import os
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

In [ ]:
# Tên thư mục nơi bạn đã lưu adapter LoRA sau fine-tuning
adapter_dir = "lora_model"

# Tên file GGUF đầu ra
gguf_file = "llama-3.1-8b-vihealthqa.gguf"


In [ ]:

# Kiểm tra xem llama.cpp đã được biên dịch chưa, nếu chưa thì biên dịch
if not os.path.exists("llama.cpp"):
    print("Không tìm thấy llama.cpp. Đang tải và biên dịch...")
    !git clone --recursive https://github.com/ggerganov/llama.cpp
    !cd llama.cpp && make clean && make all -j
    print("Đã biên dịch llama.cpp thành công.")

# Tải adapter LoRA đã fine-tuning của bạn
print(f"Đang tải adapter LoRA từ thư mục '{adapter_dir}'...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=adapter_dir,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    is_adapter=True,
)
print("Đã tải mô hình thành công.")

# Chuyển đổi và xuất file GGUF
print(f"Bắt đầu chuyển đổi mô hình sang định dạng GGUF...")
model.save_pretrained_gguf(gguf_file, tokenizer=tokenizer, llama_cpp_path="llama.cpp")
print(f"Đã xuất file GGUF thành công: {gguf_file}")


### Test với môi trường không có GPU

In [ ]:
# Cài đặt ctransformers
%pip install -q ctransformers

In [ ]:
from ctransformers import AutoModelForCausalLM

# Tải mô hình GGUF đã xuất
try:
    llm = AutoModelForCausalLM.from_pretrained(
        gguf_file,
        model_file=gguf_file,
        model_type="llama",
        gpu_layers=0 # Đảm bảo chạy trên CPU
    )
    print("Đã tải mô hình GGUF thành công.")
except Exception as e:
    print(f"Lỗi khi tải mô hình GGUF: {e}")
    llm = None

if llm:
    def create_prompt(question):
        return f"""### Question:
{question}

### Answer:
"""

    # Vòng lặp hỏi đáp để test
    print("\n" + "="*50)
    print("Bắt đầu trò chuyện với mô hình GGUF. Gõ 'exit' để thoát.")
    print("Lưu ý: Do chạy trên CPU, tốc độ trả lời có thể chậm hơn.")
    print("="*50)

    while True:
        question = input("Bạn có câu hỏi nào về sức khỏe không? ")
        if question.lower() == 'exit':
            break

        prompt = create_prompt(question)

        # Tạo câu trả lời
        generated_answer = ""
        for token in llm(prompt, stream=True):
            generated_answer += token

        # Xử lý kết quả trả về
        if '### Answer:' in generated_answer:
            generated_answer = generated_answer.split('### Answer:')[1].strip().replace('EOS', '')

        print("\n[Mô hình GGUF trả lời]:")
        print(generated_answer)
        print("-" * 50)



### tải mô hình (tùy chọn)

In [ ]:
print("\n--- Hướng dẫn tải file về máy local ---")
print(f"File GGUF đã được lưu: {gguf_file}")
print("Bạn có thể tải file này về máy tính bằng cách sử dụng giao diện Colab hoặc chạy lệnh sau:")

from google.colab import files
files.download(gguf_file)